In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import Estimate
import figure 
from EconModel import cpptools

from LimitedCommitmentModel import LimitedCommitmentModelClass

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)


plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

SAVE = False

# c++ settings

do_compile = True
threads = 20
T = 20
#cpptools.setup_nlopt(folder='cppfuncs/', do_print = True) #install nlopt

# Solve the model

In [2]:

specs = {}




#specs.update({f'model_NC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'threads':threads,'bargaining':2}}})
specs.update({f'model_FC':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': False, 'threads':threads,'bargaining':0}}})
#specs.update({f'model_LC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK':False , 'threads':threads,'bargaining':1}}})
#specs.update({f'test':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': False, 'threads':threads,'bargaining':0}}})

#specs.update({f'model_FC_doHK':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': True, 'threads':threads,'bargaining':0}}})
#specs.update({f'model_LC_doHK':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': True , 'threads':threads,'bargaining':1}}}) #ser ikke godt ud!
#specs.update({f'model_NC_doHK':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': True, 'threads':threads,'bargaining':2}}})

print(specs)



# solve different models
models = {}
for m,(name,spec) in enumerate(specs.items()):
    print(f'{name} loading...',end='')
    
    # setup model
    models = LimitedCommitmentModelClass(name=name,par=spec['par'])
    models.spec = spec

    compile_now = True if do_compile & (m==0) else False
    models.link_to_cpp(force_compile=compile_now)
    
    print(' solving...')
    %time models.solve() 
    
    #Save the data 
    #T = models.par.T
    #print(' saving...')
    #models.sim.init_love[:] = 0.2
    #models.sim.init_A[:] = 2.0 
    #np.random.seed(models.par.seed)
    #data1 = Estimate.create_data(models,start_p = 1, end_p = T-1, to_xl = True, name_xl = name)
    #print(f'Couple {np.mean(models.sim.couple,0)}')
    #print(f'HKw {np.mean(models.sim.Kw,0)}')
    #print(f'HKm {np.mean(models.sim.Km,0)}')
    #print(f'Laborw {np.mean(models.sim.labor_w,0)}')
    #print(f'Laborm {np.mean(models.sim.labor_m,0)}')
    #print(f'Asset {np.nanmean(models.sim.A,0)}')

    #var_list = ('labor_w', 'labor_m', 'cons_w', 'cons_m', 'Vw', 'Vm')
    #t_list = (1,9)
    #i_A = 250
    #i_HK = 1
    #i_HKw = i_HK
    #i_HKm = i_HK
    #i_Z = 0
    #i_Zw = i_Z
    #i_Zm = i_Z
    #i_P = 5
    #i_L = 5
    #figure.figure_single(models, var_list, t_list, i_A, i_Z, i_HK, model_name = name)
    #figure.figure_couple(models, var_list, t_list, i_A, i_Zw, i_Zm, i_HKw, i_HKm, i_L, i_P, sol_type = 'couple', model_name = name)
    #figure.figure_couple(models, var_list, t_list, i_A, i_Zw, i_Zm, i_HKw, i_HKm, i_L, i_P, sol_type = 'remain_couple', model_name = name)



    



#model = models['model FC, NO_HK ']
#model.sim.init_love[:] =0.2
##model.simulate()
#print('HK')*
#print(f'Couple {np.mean(model.sim.couple,0)}')

#print(f'Laborw {np.mean(model.sim.labor_w,0)}')
#print(f'Laborm {np.mean(model.sim.labor_m,0)}')
#print(f'Consumption W {np.mean(model.sim.cons_w,0)}')
#print(f'Consumption M {np.mean(model.sim.cons_m,0)}')
#print(f'Asset {np.nanmean(model.sim.A,0)}')
#print(f'HKw {np.mean(model.sim.Kw,0)}')
#print(f'HKm {np.mean(model.sim.Km,0)}')
#temp = model.sim.power
#I = model.sim.couple<1
#nan = np.zeros(I.shape)
#nan[I] = np.nan
#temp = np.nanmean(temp + nan,axis=0)
#print(f'Power {temp}')


{'test': {'latexname': 'limited', 'par': {'T': 20, 'do_HK': False, 'threads': 20, 'bargaining': 0}}}
test loading... solving...
CPU times: total: 8min 51s
Wall time: 51.7 s


In [6]:

models.sim.init_love[:] = 0.2
models.sim.init_A[:] = 2.0
np.random.seed(models.par.seed)

no_sim = 50

wald_FC_w_pval = np.nan + np.ones(no_sim) 
wald_NC_w_pval = np.nan + np.ones(no_sim)
wald_FC_m_pval = np.nan + np.ones(no_sim)
wald_NC_m_pval = np.nan + np.ones(no_sim)


wald_FC_w_fval = np.nan + np.ones(no_sim) 
wald_NC_w_fval = np.nan + np.ones(no_sim)
wald_FC_m_fval = np.nan + np.ones(no_sim)
wald_NC_m_fval = np.nan + np.ones(no_sim)


for i in range(no_sim):
    data = Estimate.create_data(models,start_p = 1, end_p = models.par.T-1)
    data = Estimate.create_variable(data, models.par)

    _, wald_FC_w , wald_NC_w = Estimate.main_est(data,gender= "w", do_estimate_omega = False, print_reg = False)
    _, wald_FC_m , wald_NC_m = Estimate.main_est(data,gender= "m", do_estimate_omega = False, print_reg = False)

    wald_FC_w_pval[i] = wald_FC_w.pvalue
    wald_FC_m_pval[i] = wald_FC_w.pvalue
    wald_NC_w_pval[i] = wald_FC_w.pvalue
    wald_NC_m_pval[i] = wald_FC_w.pvalue
    
    wald_FC_w_fval[i] = wald_FC_w.fvalue
    wald_FC_m_fval[i] = wald_FC_w.fvalue
    wald_NC_w_fval[i] = wald_FC_w.fvalue
    wald_NC_m_fval[i] = wald_FC_w.fvalue

#TODO: 
    # CHEK AT DEN GIVER DET SAMME SOM STATA , test når jeg zoomer ind på t (jeg tror de kører et index forskelligt), nb det virker som om teste NC og FC ikke giver der samme
    # tilføj mand 
    # wald_FC, wald_NC ud til flere kørsler


c:\Users\czk481\project\LimitedCommitmentTest-1\Estimate.py:206: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['delta_omega_w'] = data['omega_w']-data['omega_w_l']
c:\Users\czk481\project\LimitedCommitmentTest-1\Estimate.py:207: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['delta_omega_m'] = data['omega_m']-data['omega_m_l']
c:\Users\czk481\project\LimitedCommitmentTest-1\Estimate.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poo

In [8]:
wald_FC_m_pval

array([6.17224607e-19, 5.44110428e-04, 3.84215384e-02, 9.33462137e-03,
       8.46940400e-02])

In [ ]:


stop
T = models.par.T
print(' saving...')
models.sim.init_love[:] = 0.2
    
models.sim.init_A[:] = 2.0 
np.random.seed(models.par.seed)
data1 = Estimate.create_data(models,start_p = 1, end_p = T-1, to_xl = True, name_xl = name)
print(f'Couple {np.mean(models.sim.couple,0)}')
print(f'HKw {np.mean(models.sim.Kw,0)}')
print(f'HKm {np.mean(models.sim.Km,0)}')
print(f'Laborw {np.mean(models.sim.labor_w,0)}')
print(f'Laborm {np.mean(models.sim.labor_m,0)}')
print(f'Asset {np.nanmean(models.sim.A,0)}')


model = models

sol = models.sol
par = models.par
sim =  models.sim

print(f'HKw max {np.max(model.sim.Kw,0)}')
print(f'HKm max {np.max(model.sim.Km,0)}')


print(f'HKw min {np.min(model.sim.Kw,0)}')
print(f'HKm min {np.min(model.sim.Km,0)}')
#print(sim.draw_Kw_temp)
print(sim.draw_Kw)

print(f'A max {np.nanmax(model.sim.A,0)}')
print(f'A min {np.nanmin(model.sim.A,0)}')
print(sim.draw_Kw_temp)

print(f'Love max {np.nanmax(model.sim.love,0)}')
print(f'Love mean {np.nanmean(model.sim.love,0)}')
print(f'Love min {np.nanmin(model.sim.love ,0)}')




print(f'laborW max {np.nanmax(model.sim.labor_w,0)}')
print(f'laborW mean {np.nanmean(model.sim.labor_w,0)}')
print(f'LaborW min {np.nanmin(model.sim.labor_w,0)}')

print(par.grid_A)

NameError: name 'stop' is not defined

c:\Users\czk481\project\LimitedCommitmentTest-1\Estimate.py:298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Shadow_value[i] = pd.qcut(data_regress[i], 10, labels = False, duplicates='raise')
c:\Users\czk481\project\LimitedCommitmentTest-1\Estimate.py:298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Shadow_value[i] = pd.qcut(data_regress[i], 10, labels = False, duplicates='raise')
c:\Users\czk481\project\LimitedCommitmentTest-1\Estimate.py:298: SettingWithCopyWarning: 
A value is trying to be set on a c

                            OLS Regression Results                            
Dep. Variable:                    y_w   R-squared:                       0.882
Model:                            OLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     745.8
Date:                Wed, 20 Mar 2024   Prob (F-statistic):               0.00
Time:                        14:25:31   Log-Likelihood:                 81540.
No. Observations:               38504   AIC:                        -1.624e+05
Df Residuals:                   38161   BIC:                        -1.595e+05
Df Model:                         342                                         
Covariance Type:              cluster                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.0304    

In [ ]:
#print(data.shape)
#print(data.info())
#print(data.head()) #preriew the head, god måde at checke om den gør det rigtige
# data = data[data.couple>0.5] #select row that meet certain critereia
#data = data[data['couple']>0.5]
#data = data.drop(columns = ['Love','util','value']) #drop unnesecary columns
#data = data.rename(columns= {'wealth': 'wealth_lag'})
#data.isnull().sum() #sum up all null values for all columns, som algorithm does not like it 
#data = data.fillna(500) Replace all null values with 500
#data[data.wealth == 0 ].head() #printer hvor dette gælder
# data = data.dropna() #drop nan! 
#sum(data.duplicated(data.columns)) #check number of duplicates
#data.describe() #desciptive statistics
#data['district'] = data['district'].replace([1,2,3,4,5],['M'.'B','C', 'Q','S'])
# pd.get_dummies(data['t'], columns = ['t'], prefix = 'd) #convert categorical to dummy
# sample = data.sample(n=10) # a random sample of 10 homes


NameError: name 'stop' is not defined

Result, w
                            OLS Regression Results                            
Dep. Variable:                    y_w   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.480
Method:                 Least Squares   F-statistic:                     425.1
Date:                Wed, 20 Mar 2024   Prob (F-statistic):               0.00
Time:                        10:24:45   Log-Likelihood:                 89570.
No. Observations:               89314   AIC:                        -1.791e+05
Df Residuals:                   89283   BIC:                        -1.788e+05
Df Model:                          30                                         
Covariance Type:              cluster                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   

c:\Users\czk481\Anaconda3\lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 13, but rank is 11
  warnings.warn('covariance of constraints does not have full '
c:\Users\czk481\Anaconda3\lib\site-packages\statsmodels\base\model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


In [ ]:
Shadow_value = data[['t','idx']]
#data_test['earbubgs'] = pd.qcut(data['log_wealth'], 10, labels = False) 
cat = ['log_earnings_w', 'log_earnings_m','log_earnings_w_l', 'log_earnings_m_l', 'inc_share_w', 'inc_share_w_l','log_wealth', 'log_wealth_l']


#NB SKAL EVT VÆRE FORWARD I STEDET 
for i in cat:
    Shadow_value[i] = pd.qcut(data[i], 10, labels = False, duplicates='raise') 

Shadow_value['earnings_w_m'] = Shadow_value['log_earnings_w'].astype('str') + '_' + Shadow_value['log_earnings_m'].astype('str')
Shadow_value['earnings_w_m_l'] = Shadow_value['log_earnings_w_l'].astype('str') + '_' + Shadow_value['log_earnings_m_l'].astype('str')
Shadow_value['inc_share_n_l'] = Shadow_value['inc_share_w_l'].astype('str') + '_' + Shadow_value['inc_share_w'].astype('str')
Shadow_value['wealth_n_l'] = Shadow_value['log_wealth'].astype('str') + '_' + Shadow_value['log_wealth_l'].astype('str')

Shadow_value = pd.get_dummies(Shadow_value, columns=['earnings_w_m', 'earnings_w_m_l', 'inc_share_n_l','wealth_n_l' ], dtype = float)


#Drop if less thatn two
Shadow_value = Shadow_value.loc[:,(Shadow_value.sum()>2 )]
Shadow_value = Shadow_value.drop(columns = ['t','idx','log_earnings_w', 'log_earnings_m','log_earnings_w_l', 'log_earnings_m_l', 'inc_share_w', 'inc_share_w_l','log_wealth', 'log_wealth_l'])



Shadow_value.info() 
Shadow_value.sum().sort_values()


C:\Users\czk481\AppData\Local\Temp\2\ipykernel_11024\2539133232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Shadow_value[i] = pd.qcut(data[i], 10, labels = False, duplicates='raise')
C:\Users\czk481\AppData\Local\Temp\2\ipykernel_11024\2539133232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Shadow_value[i] = pd.qcut(data[i], 10, labels = False, duplicates='raise')
C:\Users\czk481\AppData\Local\Temp\2\ipykernel_11024\2539133232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a co

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38504 entries, 0 to 38503
Columns: 229 entries, earnings_w_m_0_0 to wealth_n_l_9_9
dtypes: float64(229)
memory usage: 67.3 MB


earnings_w_m_l_3_1       3.0
wealth_n_l_4_8           3.0
inc_share_n_l_1_7        3.0
wealth_n_l_5_9           4.0
wealth_n_l_6_2           4.0
                       ...  
wealth_n_l_7_7        2724.0
wealth_n_l_0_0        3113.0
wealth_n_l_1_1        3137.0
wealth_n_l_8_8        3337.0
wealth_n_l_9_9        3397.0
Length: 229, dtype: float64

# Estimate the model

In [ ]:

models.sim.init_love[:] = 0.2
models.sim.init_A[:] = 2.0
np.random.seed(models.par.seed)

#data = Estimate.create_data(models,start_p = 1, end_p = models.par.T-1)
path='output/'
name_xl = 'model_FC20'
#data = Estimate.create_data(models,start_p = 1, end_p = models.par.T-1)
data = pd.read_excel(f'{path}{name_xl}.xlsx')
data = Estimate.create_variable(data, models.par)
data_w, wald_FC_w , wald_NC_w  = Estimate.main_est(data,gender= "w", do_estimate_omega = False, print_reg = True)
data_m, wald_FC_m , wald_NC_m  = Estimate.main_est(data,gender= "m", do_estimate_omega = False, print_reg = True)

#TODO: 
    # CHEK AT DEN GIVER DET SAMME SOM STATA , test når jeg zoomer ind på t (jeg tror de kører et index forskelligt), nb det virker som om teste NC og FC ikke giver der samme
    # tilføj mand 
    # wald_FC, wald_NC ud til flere kørsler

NameError: name 'models' is not defined

In [ ]:

path='output/'
name_xl = 'model_FC20'
#data = Estimate.create_data(models,start_p = 1, end_p = models.par.T-1)
data = pd.read_excel(f'{path}{name_xl}.xlsx')
data = Estimate.create_variable(data, models.par)data.describe()

,t,init_barg,log_earnings_w,log_earnings_m,log_earnings_w_l,log_earnings_m_l,inc_share_w,inc_share_w_l,log_wealth,wealth_F,...,BMI_l,BMI_l2,BMI_j,BMI_j_l,BMI_j_l2,control_part_inc_w,control_cons,delta_log_wealth,delta_log_wealth_l,delta_log_wealth_l2
count,3932.000000,3932.000000,3932.000000,3932.000000,3932.000000,3932.000000,3932.000000,3932.000000,3932.000000,3932.000000,...,3932.000000,3932.000000,3932.000000,3932.000000,3932.000000,3932.000000,3932.000000,3932.000000,3932.000000,3932.000000
mean,12.950153,0.979145,0.246708,0.237777,0.240271,0.231078,0.502189,0.502257,0.877200,2.452181,...,0.006699,0.005994,0.005817,0.003878,0.004936,0.001240,0.015926,0.012483,0.014751,0.016941
std,0.817363,0.632836,0.232127,0.232003,0.228715,0.228641,0.104396,0.102492,0.143330,0.352987,...,0.098655,0.098701,0.104134,0.097561,0.097514,0.061641,0.014359,0.029441,0.030122,0.030789
min,12.000000,0.000000,-0.701883,-0.571739,-0.701883,-0.575289,0.238852,0.238852,0.452683,1.571454,...,-0.693147,-0.693147,-0.693147,-0.693147,-0.693147,-0.244124,-0.293143,-0.099153,-0.083850,-0.079658
25%,12.000000,1.000000,0.071564,0.064073,0.072575,0.063851,0.410221,0.413256,0.777153,2.195154,...,0.000000,0.000000,0.000000,0.000000,0.000000,-0.037310,0.011946,-0.007414,-0.005671,-0.004085
50%,13.000000,1.000000,0.320668,0.305717,0.316915,0.300977,0.504953,0.505094,0.880631,2.444084,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.005042,0.016823,0.011933,0.013716,0.016197
75%,14.000000,1.000000,0.438432,0.432233,0.426222,0.421017,0.593480,0.590909,0.978179,2.686446,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.043108,0.020880,0.031602,0.034387,0.037365
max,14.000000,2.000000,0.673958,0.643478,0.604054,0.630283,0.746163,0.737355,1.275627,3.660507,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.180661,0.374230,0.146229,0.146229,0.160442


In [ ]:
do_estimate_omega = True
path='output/'
name_xl = 'model_FC20'
#data = Estimate.create_data(models,start_p = 1, end_p = models.par.T-1)
data = pd.read_excel(f'{path}{name_xl}.xlsx')
data = Estimate.create_variable(data, models.par)

gender = "w"

if gender == "w":
    spouse = "m"
else:
    spouse = "w"
    gender = "m"

#FOR WOMAN
if do_estimate_omega:
    data['wage_shock']=data[f'omega_res_{gender}']
    data['wage_shock_j']=data[f'omega_res_{spouse}']
else:
    data['wage_shock']=data['delta_omega_w']

c:\Users\czk481\project\LimitedCommitmentTest-1\Estimate.py:206: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['delta_omega_w'] = data['omega_w']-data['omega_w_l']
c:\Users\czk481\project\LimitedCommitmentTest-1\Estimate.py:207: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['delta_omega_m'] = data['omega_m']-data['omega_m_l']
c:\Users\czk481\project\LimitedCommitmentTest-1\Estimate.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poo

In [ ]:
data[['wage_shock','wage_shock_j', 'omega_res_w', 'omega_res_m']].head()

,wage_shock,wage_shock_j,omega_res_w,omega_res_m
0,-0.041005,-0.000247,-0.041005,-0.000247
1,0.029484,0.058988,0.029484,0.058988
2,0.059427,-0.051422,0.059427,-0.051422
3,-0.106067,0.115550,-0.106067,0.115550
4,0.080281,-0.026186,0.080281,-0.026186
